## main file with solutions and functions for HorizonSideRobots course

### Zero: function collecton, execute it! (and dont forget to fold it when you dont need it)

In [11]:
using HorizonSideRobots
include("Cbot.jl")

snakeFromCorner! (generic function with 4 methods)

### Task 1: fill the cross

In [12]:
"""the following function fills the cross form"""
function cross!(bot::Cbot)
    for i in range(0, 3)
        side = HorizonSide(i)
        markLine!(bot,side)
        returnSafe!(bot)
    end
    putmarker!(bot)
end

r = Cbot("default.sit")

cross!(r)

### Task 2: fill in the entire canvas

In [13]:
"""the following function fills the whole canvas"""
function fullCanvas!(bot::Cbot; returnFunction=returnLinear!)
    moveTill!(bot, South, isborder, [bot, South])
    moveTill!(bot, West, isborder, [bot, West])
    putmarker!(bot)
    while true
        """
        moving direction is needed so that if the field has an odd number of columns,
        the program does not forget to process the last column. for a square (and not a possible rectangle),
        you can always check only the borders from the bottom and right in that particular case.
        """
        movingDirection = North
        if isborder(bot, South)
            markLine!(bot, North)
            movingDirection = North
        else
            markLine!(bot, South)
            movingDirection = South
        end

        # stopping condition
        if isborder(bot, movingDirection) && isborder(bot, East) break end
        
        move!(bot, East)
        putmarker!(bot)
    end

    returnFunction(bot)

end

r = Cbot("default.sit")

fullCanvas!(r)

true

### Task 3: fill in the perimeter

In [14]:
"""the following function fills the perimeter"""
function perimeter!(r::Cbot; returnFunction=returnLinear!)
    moveToCornerSimple(r, SouthWest, 3)
    for direction in (North, East, South, West)
        markLine!(r, direction)
    end
    
    returnFunction(r)
end

r = Cbot("default.sit")

perimeter!(r)

true

### Task 4: task: fill in the tilted cross

In [15]:
"""
Its possible to tiled the cross using the same cross!() function
I reimplemented move! and markLine! with tilted direction (45 degrees left)
run the first cell after executing this one to return the original functions!

"""

counterClockwise(side::HorizonSide) = HorizonSide(mod(Int(side)+1,4))

function tiltedMove!(r::Cbot, side::HorizonSide)
    move!(r, side)
    move!(r, counterClockwise(side))
end

function markLine!(r::Cbot,side::HorizonSide)
    while !isborder(r,side) && !isborder(r, counterClockwise(side))
        tiltedMove!(r,side)
        putmarker!(r)
    end
end

r = Cbot("default.sit")

cross!(r)
# fixing reimplemented markLine
function markLine!(bot::Cbot,side::HorizonSide)
    while !isborder(bot,side) 
        move!(bot,side)
        putmarker!(bot)
    end
end

markLine! (generic function with 2 methods)

### Task 5: task: two frames

In [16]:
# Do NOT execute it not from corner, it gets context from it
function snakeFromCorner!(bot::Cbot,mark, event::Function=HorizonSideRobots.isborder, args::Array{Any}=[])
    verticalBorder = isborder(bot, North) ? North : (isborder(bot, South) ? South : nothing)
    horizontalBorder = isborder(bot, East) ? East : (isborder(bot, West) ? West : nothing)
    @assert isnothing(verticalBorder) || isnothing(horizontalBorder) == false
    vMovementDir = inverse(verticalBorder)
    hMovementDir = inverse(horizontalBorder)
    while !event(args...)
        moveTill!(bot, hMovementDir, isborder, [bot, (hMovementDir, vMovementDir), false, true]; mark=mark)
        if isborder(bot, vMovementDir) break end
        !isborder(bot, vMovementDir) && move!(bot, vMovementDir)
        hMovementDir = inverse(hMovementDir)
    end
end

function moveAround!(bot::Cbot, initialStickingDirection=North)
    secondaryDirection = turnClockwise(initialStickingDirection)
    # print(secondaryDirection)
    moveTill!(bot, inverse(secondaryDirection),inverse, [isborder, [bot, initialStickingDirection]])
    move!(bot, secondaryDirection)
    for _ in 1:4
        moveTill!(bot, secondaryDirection,inverse, [isborder, [bot, initialStickingDirection]]; mark=true)
        move!(bot, initialStickingDirection, mark=true)
        initialStickingDirection = turnCounterClockwise(initialStickingDirection)
        secondaryDirection = turnClockwise(initialStickingDirection)
    end
end

function twoFrames!(bot::Cbot, returnFunction=returnSafe!)
    moveToCornerSimple(bot, SW, 3)
    perimeter!(bot)
    moveToCornerSimple(bot, SW, 3)
    snakeFromCorner!(bot, false, isborder, [bot, North])
    moveAround!(bot, North)
    returnFunction(bot)
end

r = Cbot("./Situations/MoveAround.sit")

twoFrames!(r)

true

## Task 6: Perimeter again a)

In [17]:
r = Cbot("Situations/6.sit")
moveToCornerInf(r, SW)
perimeter!(r;returnFunction=returnSafe!)

true

## Task 6: Perimeter again b)

In [18]:
function perimeter!(r::Cbot, event::Function, args;returnFunction=returnSafe!)
    moveToCornerInf(r, SouthWest)
    for direction in (North, East, South, West)
        markLine!(r, direction, event, args)
    end
    
    returnFunction(r)
end

r = Cbot("Situations/6.sit")

perimeter!(r,bot -> (bot.rx == 0 || bot.ry == 0), r)

true

## Task 7: Find the gap

In [19]:
function linearSearch(bot::Cbot,startingDirection::HorizonSide,  event::Function, args...)
    currentDirection = startingDirection
    ampl = 1
    while !event(args...)
        move!(bot, currentDirection, ampl)
        currentDirection = inverse(currentDirection)
        ampl += 1
    end
end

r = Cbot("Situations/7.sit")

linearSearch(r, West, inverse, isborder, r, North)

## Task 8: Find the marker

In [20]:
function ouroboros(bot::Cbot, diagonal::Diagonal, event::Function, args)
    currentDirection, secondaryDirection = unpackDiagonal(diagonal)
    rotatingFunction = secondaryDirection == turnClockwise(currentDirection) ? turnClockwise : turnCounterClockwise
    ampl = 0
    while !event(args)
        move!(bot, currentDirection, 1+div(ampl, 2), event, args)
        ampl += 1
        currentDirection=rotatingFunction(currentDirection)
    end
end

r = Cbot("./Situations/8.sit")

ouroboros(r, NW, ismarker, r)

## Task 9 and 13: Chessboard

In [21]:
using HorizonSideRobots
include("Cbot.jl")


function snakeFromCorner!(bot::Cbot, event1::Function, args1, event2::Function, args2)
    verticalBorder = isborder(bot, North) ? North : (isborder(bot, South) ? South : nothing)
    horizontalBorder = isborder(bot, East) ? East : (isborder(bot, West) ? West : nothing)
    @assert isnothing(verticalBorder) || isnothing(horizontalBorder) == false
    vMovementDir = inverse(verticalBorder)
    hMovementDir = inverse(horizontalBorder)
    while !event1(args1...)
        moveTill!(bot, hMovementDir, isborder, [bot,hMovementDir], event2, [args2])
        hMovementDir = inverse(hMovementDir)
        !isborder(bot, vMovementDir) && move!(bot, vMovementDir)
    end
end

r = Cbot("default.sit")
moveToCornerInf(r, SW)
snakeFromCorner!(r, cornered, [r, NE], bot -> (mod(bot.rx + bot.ry, 2)==0 ), r)